# DATA PRE TREATMENT

Let's start by importing all our **Libraries**

In [1]:
# IMPORT
## Actually uses
import sys
import os
import pandas as pd
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
## Actually not uses 
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from itertools import combinations
from itertools import chain
%matplotlib inline

## Importing the Dataset

In [2]:
# PATHS
DATA_PATH = os.path.join("./data")
DATA_NAME_CSV = "CyberTroll.csv"

Extraction of our data and transformation into DataFrame with **pandas**

In [3]:
#Transforming the .csv file into a DataFrame
def load_data(data_path=DATA_PATH,data_name_csv=DATA_NAME_CSV):
        # Path to file mushrooms.csv
        csv_path = os.path.join(data_path,data_name_csv)
        # Object DATA_FRAME
        return pd.read_csv(csv_path)

# execution
data_frame = load_data()

Let's look at what this data looks like

In [4]:
data_frame.head()

,content,annotation/notes,annotation/label/0,extras,metadata/first_done_at,metadata/last_updated_at,metadata/sec_taken,metadata/last_updated_by,metadata/status,metadata/evaluation
0,Get fucking real dude.,NaN,1,NaN,1527503426000,1527503426000,0,jI67aE5hwwdh6l16bcfFVnpyREd2,done,NONE
1,She is as dirty as they come and that crook R...,NaN,1,NaN,1527503426000,1527503426000,0,jI67aE5hwwdh6l16bcfFVnpyREd2,done,NONE
2,why did you fuck it up. I could do it all day ...,NaN,1,NaN,1527503426000,1527503426000,0,jI67aE5hwwdh6l16bcfFVnpyREd2,done,NONE
3,Dude they dont finish enclosing the fucking sh...,NaN,1,NaN,1527503426000,1527503426000,0,jI67aE5hwwdh6l16bcfFVnpyREd2,done,NONE
4,WTF are you talking about Men? No men thats no...,NaN,1,NaN,1527503426000,1527503426000,0,jI67aE5hwwdh6l16bcfFVnpyREd2,done,NONE


Quick check of the quantity of samples and the diversity of cases

In [5]:
data_frame.describe() 

,annotation/notes,annotation/label/0,extras,metadata/first_done_at,metadata/last_updated_at,metadata/sec_taken
count,0.0,20001.000000,0.0,2.000100e+04,2.000100e+04,20001.0
mean,NaN,0.391080,NaN,1.527503e+12,1.527503e+12,0.0
std,NaN,0.488005,NaN,2.717817e+04,2.717817e+04,0.0
min,NaN,0.000000,NaN,1.527503e+12,1.527503e+12,0.0
25%,NaN,0.000000,NaN,1.527503e+12,1.527503e+12,0.0
50%,NaN,0.000000,NaN,1.527503e+12,1.527503e+12,0.0
75%,NaN,1.000000,NaN,1.527503e+12,1.527503e+12,0.0
max,NaN,1.000000,NaN,1.527504e+12,1.527504e+12,0.0


Check if there is a null value and all the indexes we have

In [6]:
data_frame.isnull().sum()

content                         0
annotation/notes            20001
annotation/label/0              0
extras                      20001
metadata/first_done_at          0
metadata/last_updated_at        0
metadata/sec_taken              0
metadata/last_updated_by        0
metadata/status                 0
metadata/evaluation             0
dtype: int64

**annotation/notes**,**extras** is composed of null values so its implication in a classification algorithm is not relevant.

In [7]:
data_frame = data_frame.drop(['annotation/notes','extras'],axis=1)

Let's look at all the unique choices we have in all indexes, if there is an index that always has the same value we can remove it.

In [8]:
for index in data_frame.columns:
        print(str(index) + " : " + str(data_frame[index].unique()))

content : ['Get fucking real dude.'
 "She is as dirty as they come  and that crook Rengel  the Dems are so fucking corrupt it's a joke. Make Republicans look like  ..."
 "why did you fuck it up. I could do it all day too. Let's do it when you have an hour. Ping me later to sched writing a book here."
 ... 'hahahahaha >:) im evil mwahahahahahahahahaha'
 'What&;s something unique about Ohio? :)'
 'Who is the biggest gossiper you know?']
annotation/label/0 : [1 0]
metadata/first_done_at : [1527503426000 1527503427000 1527503428000 1527503429000 1527503430000
 1527503431000 1527503432000 1527503433000 1527503434000 1527503435000
 1527503436000 1527503437000 1527503438000 1527503439000 1527503440000
 1527503441000 1527503442000 1527503443000 1527503444000 1527503445000
 1527503446000 1527503447000 1527503448000 1527503449000 1527503450000
 1527503451000 1527503452000 1527503453000 1527503454000 1527503455000
 1527503456000 1527503457000 1527503458000 1527503459000 1527503460000
 15275034610

**metadata/sec_taken**,**metadata/last_updated_by**,**metadata/status**,**metadata/evaluation** is composed of a single variable so its implication in a classification algorithm is not relevant.

In [9]:
data_frame = data_frame.drop(['metadata/sec_taken','metadata/last_updated_by','metadata/status','metadata/evaluation'],axis=1)

**metadata/last_updated_at**,**metadata/first_done_at** is composed of timestamp so it's implication in not relevant.

In [10]:
data_frame = data_frame.drop(['metadata/last_updated_at','metadata/first_done_at'],axis=1)

In [11]:
data_frame.head()

,content,annotation/label/0
0,Get fucking real dude.,1
1,She is as dirty as they come and that crook R...,1
2,why did you fuck it up. I could do it all day ...,1
3,Dude they dont finish enclosing the fucking sh...,1
4,WTF are you talking about Men? No men thats no...,1


Rename columns for easier use:

In [12]:
data_frame = data_frame.rename(columns={"content" : "sentence","annotation/label/0" :"label"})

## Text Preprocessing

Text may contain numbers, special characters, and unwanted spaces. We will remove all the special characters, numbers, and unwanted spaces from our text.

In [13]:
# Lemmatizer 
stemmer = WordNetLemmatizer()
# Function to apply on our sentence
def converterSentence(text):
    # Remove all the special characters
    text = re.sub(r'\W', ' ', text)
    # remove all single characters
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    # Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    # Substituting multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    # Converting to Lowercase
    text = text.lower()
    # Lemmatization
    text = text.split()
    text = [stemmer.lemmatize(word) for word in text]
    text = ' '.join(text)
    return text

In [14]:
data_frame['sentence']= data_frame.sentence.apply(converterSentence)
data_frame.sentence

0                                    get fucking real dude
1        she is a dirty a they come and that crook reng...
2        why did you fuck it up could do it all day too...
3        dude they dont finish enclosing the fucking sh...
4        wtf are you talking about men no men thats not...
                               ...                        
19996    dont but what is complaining about it going to do
19997    bahah yeah totally just gonna get pissed at yo...
19998             hahahahaha im evil mwahahahahahahahahaha
19999                     what something unique about ohio
20000                 who is the biggest gossiper you know
Name: sentence, Length: 20001, dtype: object

# Analysing Data

Different approaches exist to convert text into the corresponding numerical form. But we are going to use Bag of Words Model because it is the most efficient.

**Default Vectorizer** :

In [15]:
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(data_frame.sentence)
Y = data_frame.label

The bag of words approach works fine for converting text to numbers. However, it has one drawback. It assigns a score to a word based on its occurrence in a particular document. It doesn't take into account the fact that the word might also be having a high frequency of occurrence in other documents as well. TFIDF resolves this issue by multiplying the term frequency of a word by the inverse document frequency. 

In [16]:
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [18]:
# Simple Random Forest Classifier
classifier = RandomForestClassifier(n_estimators=20, random_state=0)
classifier.fit(x_train, y_train) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [19]:
y_pred = classifier.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.91      0.93      2429
           1       0.87      0.93      0.90      1572

    accuracy                           0.92      4001
   macro avg       0.91      0.92      0.92      4001
weighted avg       0.92      0.92      0.92      4001

